# Curvature approximation via angle deficit

In [ ]:
from ngsolve.meshes import MakeStructured2DMesh
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw
import random as random

Gex = CF( (1+(x-x**3/3)**2, (x-x**3/3)*(y-y**3/3), 
           (x-x**3/3)*(y-y**3/3), 1+(y-y**3/3)**2), dims=(2,2) )

K_ex = 81*(1-x**2)*(1-y**2)/(9+x**2*(x**2-3)**2+y**2*(y**2-3)**2)**2

In [ ]:
bbndtang  = specialcf.VertexTangentialVectors(2)
bbndtang1 = bbndtang[:,0]
bbndtang2 = bbndtang[:,1]

def TrigAngle(G):
    return acos( G*bbndtang1*bbndtang2 / sqrt(G*bbndtang1*bbndtang1) / sqrt(G*bbndtang2*bbndtang2) )

def ComputeGaussCurv(mesh, G):
    fesCC = HCurlCurl(mesh, order=0)
    gfG = GridFunction(fesCC)
    gfG.Set(G, dual=True, bonus_intorder=5)
        

    fesH = H1(mesh, order=1, dirichlet=".*")
    u,v = fesH.TnT()

    f = LinearForm(fesH)
    f += v*(TrigAngle(Id(2))-TrigAngle(gfG))*dx(element_vb=BBND)

    gfK = GridFunction(fesH)
    
    M = BilinearForm(sqrt(Det(gfG))*u*v*dx, symmetric=True)
    M.Assemble()
    f.Assemble()
        
    gfK.vec.data = M.mat.Inverse(fesH.FreeDofs(),inverse="sparsecholesky")*f.vec
                        
    Draw(gfK, mesh, "K", min=0,max=1, deformation=True)
    
    errl2  = sqrt(Integrate( (gfK-K_ex)**2, mesh ))

    return (errl2, fesH.ndof)

In [ ]:
def UnstructuredMesh(i):
    ngmesh = unit_square.GenerateMesh(maxh=1/2**i)
    for pnts in ngmesh.Points():
        px,py = pnts[0],pnts[1]
        if abs(px) > 1e-8 and abs(px-1) > 1e-8 and abs(py) > 1e-8 and abs(py-1) > 1e-8:
            px += random.uniform(-1/2**(i+2),1/2**(i+2))
            py += random.uniform(-1/2**(i+2),1/2**(i+2))
        pnts[0] = 2*px-1
        pnts[1] = 2*py-1
    ngmesh.Update()
    return Mesh(ngmesh)

def StructuredMesh(i):
    return MakeStructured2DMesh(quads=False,nx=2**i,ny=2**i, mapping = lambda x,y : (2*x-1,2*y-1))

In [ ]:
errl2 = []
ndof  = []

with TaskManager():
    for i in range(2,7):
        mesh = StructuredMesh(i)
        #mesh = UnstructuredMesh(i)
        errl,dof = ComputeGaussCurv(mesh, Gex)
        errl2.append(errl)
        ndof.append(dof)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(ndof, errl2 , '-x', label="$\|K_h-K_{\mathrm{ex}}\|_{L^2}$")
plt.plot(ndof, [dof**(-1/2) for dof in ndof], '-' , color="k", label="$O(h)$")
plt.plot(ndof, [dof**(-2/2) for dof in ndof], '--', color="k", label="$O(h^2)$")

plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.show()